<a href="https://colab.research.google.com/github/liuxiao916/Dive_into_DL_PyTorch/blob/main/4_2_%E6%A8%A1%E5%9E%8B%E5%8F%82%E6%95%B0%E7%9A%84%E8%AE%BF%E9%97%AE%E3%80%81%E5%88%9D%E5%A7%8B%E5%8C%96%E5%92%8C%E5%85%B1%E4%BA%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.2 模型参数的访问、初始化和共享

此节参考：https://github.com/ShusenTang/Dive-into-DL-PyTorch/blob/master/docs/chapter04_DL_computation/4.2_parameters.md

In [1]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()


Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


### 4.2.1 访问模型参数

In [2]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [3]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [4]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass
    
n = MyModel()
for name, param in n.named_parameters():
    print(name)


weight1


In [5]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad) 
Y.backward()
print(weight_0.grad)

tensor([[ 0.3269, -0.4875, -0.0261, -0.1253],
        [ 0.2044, -0.1036,  0.4002, -0.1182],
        [ 0.3243,  0.3444,  0.1118, -0.3129]])
None
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0322, -0.0336, -0.0211, -0.0212],
        [-0.0199, -0.0208, -0.0131, -0.0131]])


In [6]:
for param in net[0].parameters():
    print(param)

Parameter containing:
tensor([[ 0.3269, -0.4875, -0.0261, -0.1253],
        [ 0.2044, -0.1036,  0.4002, -0.1182],
        [ 0.3243,  0.3444,  0.1118, -0.3129]], requires_grad=True)
Parameter containing:
tensor([0.1684, 0.2214, 0.2470], requires_grad=True)


In [7]:
net[0].weight

Parameter containing:
tensor([[ 0.3269, -0.4875, -0.0261, -0.1253],
        [ 0.2044, -0.1036,  0.4002, -0.1182],
        [ 0.3243,  0.3444,  0.1118, -0.3129]], requires_grad=True)

### 4.2.2 初始化模型参数

In [8]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[ 0.0013,  0.0050, -0.0097,  0.0038],
        [-0.0056, -0.0119, -0.0095,  0.0139],
        [-0.0213,  0.0011,  0.0146, -0.0134]])
2.weight tensor([[ 0.0200, -0.0158,  0.0161]])


In [9]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 4.2.3 自定义初始化方法

In [10]:
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)


In [11]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)


0.weight tensor([[ 0.0000,  7.3752, -0.0000, -0.0000],
        [-6.4828,  0.0000,  0.0000,  9.5461],
        [ 9.6620,  0.0000,  0.0000,  8.8448]])
2.weight tensor([[-8.1967,  0.0000, -0.0000]])


In [12]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)


0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 4.2.4 共享模型参数

In [13]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)


Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [14]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [15]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad) 

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
